# 2017年11月15日20:46:13
# 利用DenseNet结构改进原CNN网络

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pickle
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from cumt_variant import densenet
sess=tf.InteractiveSession()
init=tf.global_variables_initializer()

In [ ]:
#cumt rgb 相片均值
cumt_picmean=[103.939, 116.779, 123.68]
cumt_data=np.load('cumt_data.pkl')
index=np.arange(cumt_data['data'].shape[0])
np.random.shuffle(index)
data=cumt_data['data'][index]
label=cumt_data['labels'][index]
del cumt_data
resize_img=[]
for i in range(data.shape[0]):
    img=data[i]+cumt_picmean
    img=cv2.resize(img.astype('uint8'),(128,128))
    resize_img.append(img.astype('uint8'))
    if i%10000==0:
        print (i)
data=None
resize_img=np.array(resize_img).astype('uint8')

**修改DenseNet参数，希望能够减小参数，希望能用1k的数据集训练出一个ACC接近1的网络**
* ori:  k=12,  L=5,  dense block=4,  theta=0.5   ->  0.197 million,  acc=0.95
* 1: k=12 ,L=4 ,dense block=4,theta=0.5  ->0.146 million,acc=0.93
* 2: k=8 ,L=5 ,dense block=4,theta=0.5   ->0.089 million,acc=0.85
* 3: k=12 ,L=5 ,dense block=3,theta=0.5   ->0.065 million,acc=0.89
* 4: k=12 ,L=5 ,dense block=4,theta=0.3   ->0.166 million,acc=0.925
* 5: k=8 ,L=4 ,dense block=3,theta=0.3,reg=le-3   ->0.042 million,?acc=0.9
* 6: k=8 ,L=4 ,dense block=3,theta=0.3,reg=le-3,drop out=0.5   ->0.042 million,acc=0.9
* 7: k=8 ,L=4 ,dense block=3,theta=0.3,reg=4e-4,drop out=0.5,data size =2048   ->0.042 million,?acc=0.95

*总结：*
* *减少L,theta对准确率的影响最小，但是减少的参数也少*
* *减小K,dense block数量能大量的减小参数，但是同时也会比较大程度的影响ACC*
* *增加正则和drop out能稍微增加一点准确率，但不是关键的因素*


In [ ]:
log_name='cumtdata/densenet_samllv7_conv'
with tf.variable_scope('Placehloder'):
    X=tf.placeholder(dtype=tf.float32,shape=[None,128,128,3],name='X')
    Y=tf.placeholder(dtype=tf.float32,shape=[None,10],name='Y')
    bn_train=tf.placeholder(dtype=tf.bool,name='BN_FLAG')
    LR=tf.placeholder(dtype=tf.float32,name='lr')
    DROPOUT=tf.placeholder(dtype=tf.float32,name='DROPOUT')
with tf.name_scope('DenseNet'):
    model=densenet(Images=X,dropout=DROPOUT,bn_istraining=bn_train,K=8,L=4,theta=0.3,output_class=10,sess=sess,denseblock_num=3)
    y_score=model.y_score
with tf.name_scope('LOSS'):
    LOSS=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_score,labels=Y))+4e-4*(model.get_l2loss())
    tf.summary.scalar('loss',LOSS)
with tf.name_scope('TRAIN'):
    TRAIN=tf.train.AdamOptimizer(LR).minimize(LOSS)
with tf.name_scope('ACCURACY'):
    acc_count=tf.equal(tf.arg_max(y_score,1),tf.arg_max(Y,1))
    ACCURACY=tf.reduce_mean(tf.cast(acc_count,tf.float32))
    tf.summary.scalar('acc',ACCURACY)
bn_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)

writer_te=tf.summary.FileWriter(r'./mylog/'+log_name+r'/test')
writer_tr=tf.summary.FileWriter(r'./mylog/'+log_name+r'/train')
writer_val=tf.summary.FileWriter(r'./mylog/'+log_name+r'/val')
merge=tf.summary.merge_all()



In [ ]:
def count_trainable_params():
    total_parameters = 0
    for variable in tf.trainable_variables():
        shape = variable.get_shape()
        variable_parametes = 1
        for dim in shape:
            variable_parametes *= dim.value
        total_parameters += variable_parametes
    print("Total training params: %.5f Million,%.5f Mb" % (total_parameters / 1e6,total_parameters* 4/ (1024*1024)))
count_trainable_params()

In [ ]:
data_size=1024*2

data=resize_img

index=np.arange(data.shape[0])
np.random.shuffle(index)
tr_index=index[:data_size]
te_index=index[data_size:]
# tr_index=index[:int(data.shape[0]*0.9)]
# te_index=index[int(data.shape[0]*0.9):]
tr_data,tr_label=data[tr_index],label[tr_index]
te_data,te_label=data[te_index],label[te_index]
del data
del label
del resize_img
print(tr_data.shape,te_data.shape)


val_d=np.load('valimg1k_1109.pkl')
val_data=[]
for v in range(val_d['data'].shape[0]):
    img_=cv2.resize(val_d['data'][v],(128,128))
    
    val_data.append(img_)
val_data=np.array(val_data)
val_label=val_d['label']
del val_d
print(val_data.shape)

In [ ]:
sess.run(tf.global_variables_initializer())
saver=tf.train.Saver()

In [ ]:
#开始正式训练
batchsize=128
best_acc=0
lr_=1e-3
dp_=0.5
test_acc_his=[]
for i in range(1,1000*6*12):
    mask=np.random.choice(tr_data.shape[0],batchsize,replace=False)
    x_,y_=tr_data[mask]-cumt_picmean,tr_label[mask]
    feed_dict={X:x_,Y:y_,bn_train:True,LR:lr_,DROPOUT:dp_}
    sess.run([TRAIN,bn_ops],feed_dict=feed_dict)
    if i%10==0:
        feed_dict={X:x_,Y:y_,bn_train:False,DROPOUT:1.}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_tr.add_summary(m_,i)
        print('epoch:{},train loss:{},train accuracy:{}'.format(i,loss_,acc_))
    if i%20==0:
        mask=np.random.choice(te_data.shape[0],batchsize,replace=False)
        x_,y_=te_data[mask]-cumt_picmean,te_label[mask]
        feed_dict={X:x_,Y:y_,bn_train:False,DROPOUT:1.}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_te.add_summary(m_,i)
        test_acc_his.append(acc_)
        print('--epoch:{},test loss:{},test accuracy:{},mean acc:{}'.format(i,loss_,acc_,np.mean(test_acc_his[-20:])))
    if i%30==0:
        mask=np.random.choice(val_data.shape[0],batchsize,replace=False)
        x_,y_=val_data[mask]-cumt_picmean,val_label[mask]
        feed_dict={X:x_,Y:y_,bn_train:False,DROPOUT:1.}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        best_acc=max(best_acc,acc_)
        writer_val.add_summary(m_,i)
        print('@@epoch:{},val loss:{},val accuracy:{},best val acc:{}'.format(i,loss_,acc_,best_acc))
    if i%400==0:
        lr_=max(lr_*0.8,5e-5)
        print('$$$ lr change:{}'.format(lr_))

In [ ]:
model.save_network_weight('densenet_dump/smallmodel_acc964.pkl',sess=sess)

In [ ]:
saver.save(sess,'DenseNet/model_578',global_step=i)

## 使用CUMT训练好的模型进行迁移 学习

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pickle
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from cumt_variant import densenet
cumt_picmean=[103.939, 116.779, 123.68]
sess=tf.InteractiveSession()
init=tf.global_variables_initializer()

In [ ]:
output_class=25
log_name='densenet5X5_zeros_256'
with tf.variable_scope('Placehloder'):
    X=tf.placeholder(dtype=tf.float32,shape=[None,128,128,3],name='X')
    Y=tf.placeholder(dtype=tf.float32,shape=[None,output_class],name='Y')
    bn_train=tf.placeholder(dtype=tf.bool,name='BN_FLAG')
    LR=tf.placeholder(dtype=tf.float32,name='lr')
with tf.name_scope('DenseNet'):
    model=densenet(Images=X,bn_istraining=bn_train,K=12,L=5,theta=0.5,output_class=output_class,sess=sess)
    y_score=model.y_score
with tf.name_scope('LOSS'):
    LOSS=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_score,labels=Y))
    tf.summary.scalar('loss',LOSS)
with tf.name_scope('TRAIN'):
    TRAIN=tf.train.AdamOptimizer(LR).minimize(LOSS)
with tf.name_scope('ACCURACY'):
    acc_count=tf.equal(tf.arg_max(y_score,1),tf.arg_max(Y,1))
    ACCURACY=tf.reduce_mean(tf.cast(acc_count,tf.float32))
    tf.summary.scalar('acc',ACCURACY)
bn_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)

writer_te=tf.summary.FileWriter(r'./mylog/'+log_name+r'/test')
writer_tr=tf.summary.FileWriter(r'./mylog/'+log_name+r'/train')
writer_val=tf.summary.FileWriter(r'./mylog/'+log_name+r'/val')
merge=tf.summary.merge_all()

In [ ]:
# <tf.Variable 'Tail/conv_class/weight:0' shape=(1, 1, 57, 10) dtype=float32_ref>
# <tf.Variable 'Tail/conv_class/biases:0' shape=(10,) dtype=float32_ref>
# <tf.Variable 'Tail/conv_class/BN/beta:0' shape=(10,) dtype=float32_ref>
# <tf.Variable 'Tail/conv_class/BN/gamma:0' shape=(10,) dtype=float32_ref>
model.init_network(sess=sess,weight_addr='densenet_1115.pkl',
                   skip_layer=['Tail/conv_class/weight','Tail/conv_class/biases','Tail/conv_class/BN/beta','Tail/conv_class/BN/gamma'],
                      )

In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
#  'img4X4_1006.pkl'   'img5X5_240.pkl'
# d=np.load('img4X4_1006.pkl')
d=np.load('img5X5_950.pkl')
index_=np.arange(d['data'].shape[0])
np.random.shuffle(index_)
# tr_index=index_[:int(d['data'].shape[0]*0.8)]
# te_index=index_[int(d['data'].shape[0]*0.8):]
tr_index=index_[:256]
te_index=index_[256:]
tr_data,tr_label=d['data'][tr_index],d['label'][tr_index]
te_data,te_label=d['data'][te_index],d['label'][te_index]
print(tr_data.shape,te_data.shape)
del d

In [ ]:
#开始正式训练
batchsize=104
lr_=1e-3
for i in range(1,1000*6*12):
    mask=np.random.choice(tr_data.shape[0],batchsize,replace=False)
    x_,y_=tr_data[mask]-cumt_picmean,tr_label[mask]
    feed_dict={X:x_,Y:y_,bn_train:True,LR:lr_}
    sess.run([TRAIN,bn_ops],feed_dict=feed_dict)
    if i%10==0:
        feed_dict={X:x_,Y:y_,bn_train:False}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_tr.add_summary(m_,i)
        print('epoch:{},train loss:{},train accuracy:{}'.format(i,loss_,acc_))
    if i%20==0:
        mask=np.random.choice(te_data.shape[0],batchsize,replace=False)
        x_,y_=te_data[mask]-cumt_picmean,te_label[mask]
        feed_dict={X:x_,Y:y_,bn_train:False}
        loss_,acc_,m_=sess.run([LOSS,ACCURACY,merge],feed_dict=feed_dict)
        writer_te.add_summary(m_,i)
        print('--epoch:{},test loss:{},test accuracy:{}'.format(i,loss_,acc_))
    if i%400==0:
        lr_=max(lr_*0.5,1e-5)
        print('$$$ lr change:{}'.format(lr_))

In [ ]:
saver=tf.train.Saver()
saver.save(sess,'./model_save/Densenet_trans/model_991.ckpt',global_step=i)

In [ ]:
model.save_network_weight('dense4x4_trans991.pkl',sess)

In [ ]:
%%timeit
mask=np.random.choice(10,1,replace=False)
x_,y_=te_data[mask]-cumt_picmean,te_label[mask]
feed_dict={X:x_,Y:y_,bn_train:False}
sess.run(y_score,feed_dict=feed_dict)

## 实测 5X5分类

In [ ]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')  
def drawline(img_,line_w=1,line_color=(0,0,0),wandh_num=4):
        '''
        在图片上格子
        :param line_w: 线宽
        :param line_color: 线颜色
        :param wandh_num:  长宽线的数量
        :return:  无
        '''
        h,w=img_.shape[0],img_.shape[1]
        w_num,h_num=wandh_num,wandh_num
        h_,w_=h//h_num,w//w_num

        # 竖线 (w,h)
        for i in range(1,w_num):
            #print(i)
            cv2.line(img_,(w_*i,0),(w_*i,h),line_color,line_w)
        # 横线
        for i in range(1,h_num):
            cv2.line(img_,(0,h_*i),(w,h_*i),line_color,line_w)
        return img_

def drawblock(img,line_num,block_id=0,blockcolor=(46,218,255),blockwideth=5,show_rec='fill',rec_shape=None):
        '''
        选定九宫格，在这个格子上填充矩形表示选定这个格子
        :param img_: 图片
        :param block: 九宫格序号 0-15
        :param blockcolor: 矩形框颜色
        :param blockwideth: 框的宽度
        :return:
        '''
        h,w=img.shape[0],img.shape[1]
        w_line,h_line=line_num,line_num
        h_,w_=h//h_line,w//w_line
        cor_h=block_id//line_num
        cor_w=block_id%line_num
        sx,sy=cor_w*w_,cor_h*h_
        
        if show_rec=='fill':
        #将整个矩形填充为其他颜色
            img[sy:sy+h_,sx:sx+w_,:]=blockcolor
        elif show_rec=='rec':
            #显示矩形轮廓
            xe,ye,we,he=rec_shape
            cv2.rectangle(img,(xe,ye),(xe+we,ye+he),(255,0,0),10)
        elif show_rec=='dot':
            #标记一个小点
            roi_=img[sy:sy+h_,sx:sx+w_]
            cv2.circle(roi_,(roi_.shape[1]//2,roi_.shape[0]//2), 10, (255,128,120), -1)
        elif show_rec=='select':
            #hight light 矩形区域
            #img[sy:sy+h_,sx:sx+w_,0]=255#r
            img[sy:sy+h_,sx:sx+w_,1]=255#g
            #img[sy:sy+h_,sx:sx+w_,2]=255#b
        elif show_rec=='pick':
            #返回矩形框内容
            return img[sy:sy+h_,sx:sx+w_],(sx,sy,w_,h_)
        return img
def geteyeimg(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    faces = face_cascade.detectMultiScale(gray, 1.3,5) 
    if len(faces) !=1:
        print(len(faces))
        print('bad faces')
        return None
    for (x,y,w,h) in faces: 
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2) 
            roi_gray = gray[y:y+h, x:x+w] 
            roi_color = img[y:y+h, x:x+w] 
            #检测视频中脸部的眼睛，并用vector保存眼睛的坐标、大小（用矩形表示） 
            eyes = eye_cascade.detectMultiScale(roi_gray,scaleFactor=1.2, minNeighbors=7, minSize=(29, 29),
                                     flags=cv2.CASCADE_SCALE_IMAGE) 
            #眼睛检测 ,对于识别比较差的情况舍弃
            if len(eyes)!=2:
                print(len(eyes))
                print('bad eyes')
                return None
            if eyes[0][0]>eyes[1][0]:
                ex=eyes[1][0]
                W=eyes[0][0]-eyes[1][0]+eyes[0][2]
            else:
                ex=eyes[0][0]
                W=eyes[1][0]-eyes[0][0]+eyes[1][2]
            
            if eyes[0][1]>eyes[1][1]:
                ey=eyes[1][1]
                H=eyes[0][1]-eyes[1][1]+eyes[0][3]
            else:
                ey=eyes[0][1]
                H=eyes[1][1]-eyes[0][1]+eyes[1][3]
    
    return roi_color[ey+10:ey+H-2,ex-10:ex+W+10] if roi_color is not None else None

In [ ]:
cap=cv2.VideoCapture(0)
_,frame=cap.read()
cap.release()

In [ ]:
prob=tf.nn.softmax(y_score)

In [ ]:
e=geteyeimg(frame)

In [ ]:
e=cv2.resize(e,(128,128))[:,:,::-1].reshape((1,128,128,3))

In [ ]:
#e=get_eyeimg(frame)
prob_=sess.run(prob,feed_dict={X:e-cumt_picmean,bn_train:False})

In [ ]:
prob_

In [ ]:
import time

In [ ]:
fps=10
fsize_desk=(1920,1080)
# save video
video_d=cv2.VideoWriter('demo_desk.flv',cv2.VideoWriter_fourcc('F','L','V','1'),fps,fsize_desk)
cap=cv2.VideoCapture(0)
s_time=time.time()
tar_img=cv2.imread('test.jpg')
#设置window 为全屏
cv2.namedWindow('Capture001',cv2.WINDOW_NORMAL)
cv2.setWindowProperty('Capture001', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

s_time=time.time()
random_show=np.random.randint(0,16,1)[0]

#每N帧输出平均预测
sum_score=[]
sum_counter=0
block_id=0
ret_counter=0
right_frame_counter=0
while True:
    ret,fram=cap.read()
    if ret:
        ret_counter+=1
        #显示 内窥镜图像
        tar_img=cv2.imread('test.jpg')
        tar_img=drawline(tar_img,wandh_num=5)
        #每30帧换点
        if ret_counter%22==0:
            random_show=np.random.randint(0,16,1)[0]

        tar_img=drawblock(tar_img,line_num=5,block_id=random_show)
        cv2.imshow('Capture001',tar_img)
        e=geteyeimg(fram)
        if e is None:
            continue
        e=cv2.resize(e,(128,128))[:,:,::-1].reshape((1,128,128,3))
        y_guess=sess.run(prob,feed_dict={X:e-cumt_picmean,bn_train:False})
        sum_score.append(y_guess)
        #平均每 N 帧的预测分数
        if ret_counter%5==0:
            sum_score=np.array(sum_score)
            mean_=np.mean(sum_score,0)
            block_id=np.argmax(mean_)-1
            sum_score=[]
        if block_id<0:block_id=0
        elif block_id>25:block_id=25

        print(block_id)
        if block_id==random_show:
             right_frame_counter+=1
        tar_img=drawblock(tar_img,line_num=5,block_id=block_id,show_rec='dot')
        v_img=tar_img.copy()
        fram=cv2.resize(fram,(150,150))
        v_img[:150,-150:]=fram
        cv2.imshow('Capture001',tar_img)


#         print('{} sec per frame'.format(time.time()-s_time))
        s_time=time.time()
        video_d.write(v_img)
        if cv2.waitKey(1)&0xff==27:
            print('accuracy:{}'.format(right_frame_counter/ret_counter))
            print('out')
            break
    if (time.time()-s_time)>60*5:
        print('time out')
        break
video_d.release()
cap.release()
cv2.destroyAllWindows()


In [ ]:
video_d.release()
cap.release()
cv2.destroyAllWindows()
